In [45]:
import torch
import torchvision

import numpy as np

from torch.utils.data import Dataset


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cpu for inference


In [46]:
num_epochs = 4
batch_size = 4
learning_rate = 0.001

# Czemu .5 ?
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [47]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [48]:
model = ConvNet().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [49]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Epoch [1/4], Step [2000/12500], Loss: 2.3070
Epoch [1/4], Step [4000/12500], Loss: 2.3066
Epoch [1/4], Step [6000/12500], Loss: 2.3325
Epoch [1/4], Step [8000/12500], Loss: 2.2118
Epoch [1/4], Step [10000/12500], Loss: 2.1588
Epoch [1/4], Step [12000/12500], Loss: 2.3035
Epoch [2/4], Step [2000/12500], Loss: 2.6945
Epoch [2/4], Step [4000/12500], Loss: 2.2378
Epoch [2/4], Step [6000/12500], Loss: 1.7612
Epoch [2/4], Step [8000/12500], Loss: 1.7523
Epoch [2/4], Step [10000/12500], Loss: 2.4517
Epoch [2/4], Step [12000/12500], Loss: 1.2493
Epoch [3/4], Step [2000/12500], Loss: 2.2932
Epoch [3/4], Step [4000/12500], Loss: 2.3181
Epoch [3/4], Step [6000/12500], Loss: 1.5454
Epoch [3/4], Step [8000/12500], Loss: 1.5136
Epoch [3/4], Step [10000/12500], Loss: 2.3407
Epoch [3/4], Step [12000/12500], Loss: 1.1357
Epoch [4/4], Step [2000/12500], Loss: 2.0807
Epoch [4/4], Step [4000/12500], Loss: 2.2747
Epoch [4/4], Step [6000/12500], Loss: 1.5968
Epoch [4/4], Step [8000/12500], Loss: 1.2943
Epoc

In [50]:
for para in model.parameters():
    para.requires_grad = False
    print(para)

Parameter containing:
tensor([[[[ 0.1740,  0.1174, -0.0688, -0.0498,  0.1842],
          [ 0.0819, -0.0573, -0.0920, -0.0209,  0.1025],
          [ 0.0277, -0.0160, -0.0370, -0.1393,  0.0193],
          [-0.1004, -0.0325,  0.0035, -0.1454, -0.0457],
          [-0.1371, -0.1814, -0.0269, -0.0744, -0.0746]],

         [[-0.0751, -0.0330, -0.2144, -0.0483, -0.1803],
          [ 0.0360,  0.0222, -0.1541, -0.0910, -0.1402],
          [ 0.0343, -0.0845, -0.2101, -0.1483, -0.0717],
          [ 0.0094, -0.1629, -0.0444, -0.0772, -0.1253],
          [-0.0209, -0.0647,  0.0263,  0.0533, -0.0019]],

         [[ 0.1039, -0.0823, -0.0031, -0.0960, -0.0922],
          [ 0.1004,  0.0569, -0.1281, -0.1616,  0.1033],
          [ 0.1939,  0.0034, -0.0658,  0.0382, -0.0589],
          [ 0.1719, -0.0477, -0.1068, -0.0709,  0.1202],
          [ 0.0372, -0.0616, -0.0573, -0.0821, -0.0408]]],


        [[[ 0.0181, -0.0613,  0.0627,  0.0939,  0.0785],
          [-0.0738, -0.1738, -0.1155, -0.2324, -0.1745],
 

In [62]:
for para in model.parameters():
    para.requires_grad = True
    print(para)

Parameter containing:
tensor([[[[ 0.1740,  0.1174, -0.0688, -0.0498,  0.1842],
          [ 0.0819, -0.0573, -0.0920, -0.0209,  0.1025],
          [ 0.0277, -0.0160, -0.0370, -0.1393,  0.0193],
          [-0.1004, -0.0325,  0.0035, -0.1454, -0.0457],
          [-0.1371, -0.1814, -0.0269, -0.0744, -0.0746]],

         [[-0.0751, -0.0330, -0.2144, -0.0483, -0.1803],
          [ 0.0360,  0.0222, -0.1541, -0.0910, -0.1402],
          [ 0.0343, -0.0845, -0.2101, -0.1483, -0.0717],
          [ 0.0094, -0.1629, -0.0444, -0.0772, -0.1253],
          [-0.0209, -0.0647,  0.0263,  0.0533, -0.0019]],

         [[ 0.1039, -0.0823, -0.0031, -0.0960, -0.0922],
          [ 0.1004,  0.0569, -0.1281, -0.1616,  0.1033],
          [ 0.1939,  0.0034, -0.0658,  0.0382, -0.0589],
          [ 0.1719, -0.0477, -0.1068, -0.0709,  0.1202],
          [ 0.0372, -0.0616, -0.0573, -0.0821, -0.0408]]],


        [[[ 0.0181, -0.0613,  0.0627,  0.0939,  0.0785],
          [-0.0738, -0.1738, -0.1155, -0.2324, -0.1745],
 

In [55]:
params = model.state_dict()
params.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])

In [56]:
model.fc3.weight.requires_grad = True
for name, param in model.named_parameters():
    if param.requires_grad:print(name)

conv1.weight
conv1.bias
conv2.weight
conv2.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
fc3.weight
fc3.bias


In [63]:
for name, param in model.named_parameters():
    if param.requires_grad and 'fc3' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [60]:
for name, param in model.named_parameters():print(name, param)

conv1.weight Parameter containing:
tensor([[[[ 0.1740,  0.1174, -0.0688, -0.0498,  0.1842],
          [ 0.0819, -0.0573, -0.0920, -0.0209,  0.1025],
          [ 0.0277, -0.0160, -0.0370, -0.1393,  0.0193],
          [-0.1004, -0.0325,  0.0035, -0.1454, -0.0457],
          [-0.1371, -0.1814, -0.0269, -0.0744, -0.0746]],

         [[-0.0751, -0.0330, -0.2144, -0.0483, -0.1803],
          [ 0.0360,  0.0222, -0.1541, -0.0910, -0.1402],
          [ 0.0343, -0.0845, -0.2101, -0.1483, -0.0717],
          [ 0.0094, -0.1629, -0.0444, -0.0772, -0.1253],
          [-0.0209, -0.0647,  0.0263,  0.0533, -0.0019]],

         [[ 0.1039, -0.0823, -0.0031, -0.0960, -0.0922],
          [ 0.1004,  0.0569, -0.1281, -0.1616,  0.1033],
          [ 0.1939,  0.0034, -0.0658,  0.0382, -0.0589],
          [ 0.1719, -0.0477, -0.1068, -0.0709,  0.1202],
          [ 0.0372, -0.0616, -0.0573, -0.0821, -0.0408]]],


        [[[ 0.0181, -0.0613,  0.0627,  0.0939,  0.0785],
          [-0.0738, -0.1738, -0.1155, -0.2324

In [64]:
for name, param in model.named_parameters():
    if param.requires_grad:print(name)

fc3.weight
fc3.bias


In [65]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 2000 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Epoch [1/4], Step [2000/12500], Loss: 1.9767
Epoch [1/4], Step [4000/12500], Loss: 2.2031
Epoch [1/4], Step [6000/12500], Loss: 1.7629
Epoch [1/4], Step [8000/12500], Loss: 1.2346
Epoch [1/4], Step [10000/12500], Loss: 2.0190
Epoch [1/4], Step [12000/12500], Loss: 1.2712
Epoch [2/4], Step [2000/12500], Loss: 2.0073
Epoch [2/4], Step [4000/12500], Loss: 2.2203
Epoch [2/4], Step [6000/12500], Loss: 1.7555
Epoch [2/4], Step [8000/12500], Loss: 1.2156
Epoch [2/4], Step [10000/12500], Loss: 2.0048
Epoch [2/4], Step [12000/12500], Loss: 1.2683
Epoch [3/4], Step [2000/12500], Loss: 2.0205
Epoch [3/4], Step [4000/12500], Loss: 2.2265
Epoch [3/4], Step [6000/12500], Loss: 1.7438
Epoch [3/4], Step [8000/12500], Loss: 1.2086
Epoch [3/4], Step [10000/12500], Loss: 1.9921
Epoch [3/4], Step [12000/12500], Loss: 1.2591
Epoch [4/4], Step [2000/12500], Loss: 2.0273
Epoch [4/4], Step [4000/12500], Loss: 2.2324
Epoch [4/4], Step [6000/12500], Loss: 1.7354
Epoch [4/4], Step [8000/12500], Loss: 1.2057
Epoc